In [1]:
import numpy as np
import math

## Solution for Problem 1 and 2

In [35]:
def euler_to_rotation(a,b,c):
    R=np.matrix([[math.cos(a)*math.cos(b) , (math.cos(a)*math.sin(b)*math.sin(c))-(math.sin(a)**math.cos(c)) , (math.cos(a)*math.sin(b)*math.cos(c))+(math.sin(a)*math.sin(c))],
                [(math.sin(a)*math.cos(b)) , (math.sin(a)*math.sin(b)*math.sin(c)) + (math.cos(a)*math.cos(c)), (math.sin(a)*math.sin(b)*math.cos(c))-(math.cos(a)*math.sin(c))],
               [-1*math.sin(b) , math.cos(b)*math.sin(c) , math.cos(b)*math.cos(c)]])
    return R

In [36]:
def toDegrees(angle):
    return angle * (math.pi/180);

In [37]:
def find_sign(a,b):
    if a-b>=0:
        return 1
    else:
        return -1

In [57]:
def rotation_to_unit_quaternion(R):
    neta=0.5*math.sqrt((R.item(0,0)+R.item(1,1)+R.item(2,2)+1))
    a=find_sign(R.item(2,1),R.item(1,2))
    b=find_sign(R.item(0,2),R.item(2,0))
    c=find_sign(R.item(1,0),R.item(0,1))
    eta=np.matrix([[0.5*a*math.sqrt(R.item(0,0)-R.item(1,1)-R.item(2,2)+1)],
                     [0.5*b*math.sqrt(R.item(1,1)-R.item(2,2)-R.item(0,0)+1)],
                     [0.5*c*math.sqrt(R.item(2,2)-R.item(0,0)-R.item(1,1)+1)]])
    return neta,eta

## Solution to Problem 1

In [58]:
conversion=toDegrees(60)

In [59]:
ER=euler_to_rotation(conversion,0,0)

In [60]:
'''
neta followed by eta for unit_quaternion representation
'''

rotation_to_unit_quaternion(ER)

(0.8660254037844386, matrix([[0. ],
         [0. ],
         [0.5]]))

In [61]:
ER
'''
Rotation matrix
'''

matrix([[ 0.5      , -0.8660254,  0.       ],
        [ 0.8660254,  0.5      ,  0.       ],
        [-0.       ,  0.       ,  1.       ]])

## Solution to problem 2

In [63]:
conversion_a=toDegrees(90)
conversion_b=toDegrees(60)
conversion_c=toDegrees(22.5)

In [64]:
ER=euler_to_rotation(conversion_a,-1*conversion_b,-1*conversion_c)

In [65]:
ER
'''
Rotation Matrix
'''

matrix([[ 3.06161700e-17, -1.00000000e+00, -3.82683432e-01],
        [ 5.00000000e-01,  3.31413574e-01, -8.00103145e-01],
        [ 8.66025404e-01, -1.91341716e-01,  4.61939766e-01]])

In [66]:
'''
neta followed by eta for unit_quaternion representation
'''

rotation_to_unit_quaternion(ER)

(0.6695807158758449, matrix([[ 0.22729203],
         [-0.4662279 ],
         [ 0.53163103]]))

## Solution for Problem 3

In [68]:
R=np.matrix([[0.6533, -0.6533, -0.3827],
[-0.2706,0.2706, -0.9239],
[0.7071, 0.7071, 0.0]])

In [69]:
def rotation_to_euler(R):
    alpha=math.atan2(-1*R.item(1,0),-1*R.item(0,0))
    beta=math.atan2(-1*R.item(2,0),math.sqrt(R.item(2,1)**2+R.item(2,2)**2))
    gamma=math.atan2(-1*R.item(2,1),-1*R.item(2,2))
    return alpha,beta,gamma

In [70]:
rotation_to_euler(R)

(2.7489010456323224, -0.7853981633974483, -1.5707963267948966)

In [72]:
rotation_to_unit_quaternion(R)

(0.693523611710517, matrix([[ 0.58794132],
         [-0.39284221],
         [ 0.13793114]]))

## Solution for Problem 4

In [26]:
v,w = 0.5,-0.1
alpha_one, alpha_two, alpha_three, alpha_four, alpha_five, alpha_six= 0.2, 0.1, 0.3, 0.2, 0.1, 0.3
x, y, theta = 1, 0.75, 0.785
delta_t = 0.25

In [30]:
def sample_motion_model_velocity(u, w, x, y, theta):
    v_cap = v + np.random.normal(loc = alpha_one * abs(v) + alpha_two * abs(w))
    w_cap = w + np.random.normal(loc = alpha_three * abs(v) + alpha_four * abs(w))
    
    gamma_cap = np.random.normal(loc = alpha_five * abs(v) + alpha_six * abs(w))
    
    x_bar = x - (v_cap/w_cap) * math.sin(theta) + (v_cap/w_cap) * math.sin(theta + w_cap*delta_t)
    y_bar = y + (v_cap/w_cap)* math.cos(theta) + (v_cap/w_cap) * math.cos(theta + w_cap*delta_t)
    
    theta_bar = theta + w_cap*delta_t + gamma_cap*delta_t
    
    # x_t = np.array([x_bar, y_bar, theta_bar])
    
    return x_bar, y_bar, theta_bar

In [31]:
def motion_model_velocity(x_bar, y_bar, theta_bar, u, w, x, y, theta):
    mu = ((x-x_bar) * math.cos(theta) + (y-y_bar) * math.sin(theta))/(2 * (y-y_bar) * math.cos(theta) - (x-x_bar) * math.sin(theta))
    
    x_star = ((x + x_bar) / 2) + mu * (y-y_bar)
    y_star = ((y + y_bar) / 2) + mu * (x-x_bar)
    
    r_star = math.sqrt((x-x_star)**2 + (y-y_star)**2)
    
    delta_theta = math.atan2(y_bar - y_star , x_bar - x_star) - math.atan2(y-y_star, x-x_star)
    
    v_cap = (delta_theta / delta_t) * r_star
    w_cap = (delta_theta / delta_t)
    
    gamma_cap = ((theta_bar - theta)/ delta_t) - w_cap
    
    return v_cap, w_cap, gamma_cap   
    

In [36]:
x_bar, y_bar, theta_bar = sample_motion_model_velocity(u,w, x, y, theta)

x_t = np.array([x_bar, y_bar, theta_bar])
print (f'x_t = {x_t}')

x_t = [0.98840453 0.53155079 0.76066124]


In [38]:
v_cap, w_cap, gamma_cap = motion_model_velocity(x_bar, y_bar, theta_bar, u, w, x, y, theta)
print (f'v_cap = {v_cap} \nw_cap = {w_cap} \ngamma_cap = {gamma_cap}')

v_cap = -2.9190073259292073 
w_cap = -19.1623570113974 
gamma_cap = 19.065001969319614


In [41]:
def probability(a,b):
    return (1/math.sqrt(2*3.14*b)) * math.exp((-a**2)/2 * b)

In [46]:
print (f'v Prob: {probability(v-v_cap, alpha_one * abs(v) + alpha_two * abs(w))} \nw Prob: {probability(w-w_cap, alpha_three * abs(v) + alpha_four * abs(w))} \ngamma Prob: {probability(gamma_cap, alpha_five * abs(v) + alpha_six * abs(w))}')

v Prob: 0.6325625665304959 
w Prob: 3.731241694008559e-14 
gamma Prob: 6.843525201897466e-07


## Solution for Problem 5

In [76]:
x,y,theta = 0,0,-0.2
x_head, y_head, theta_head = 0,0,0
x_head_bar, y_head_bar, theta_head_bar = 0.4, 0.2, 0.5
alpha_one, alpha_two, alpha_three, alpha_four = 0.1, 0.2, 0.4, 0.3

In [77]:
def motion_model_odometry(x,y,theta, x_head, y_head, theta_head, x_head_bar, y_head_bar, theta_head_bar):
    d_rot1 = math.atan2(y_head_bar - y_head, x_head_bar - x_head) - theta_head
    d_trans = math.sqrt((x_head - x_head_bar)**2 + (y_head - y_head_bar) ** 2)
    d_rot2 = theta_head_bar - theta_head - d_rot1
    
    d_cap_rot1 = math.atan2(y_bar - y, x_bar - x) - theta
    d_cap_trans = math.sqrt((x-x_bar)**2 + (y-y_bar)**2)
    d_cap_rot2 = theta_bar - theta - d_cap_rot1
    
    return d_rot1, d_trans, d_rot2, d_cap_rot1, d_cap_trans, d_cap_rot2

In [78]:
d_rot1, d_trans, d_rot2, d_cap_rot1, d_cap_trans, d_cap_rot2 = motion_model_odometry(x,y,theta, x_head, y_head, theta_head, x_head_bar, y_head_bar, theta_head_bar)
print (f'd_cap_rot1 = {d_cap_rot1} \nd_cap_trans = {d_cap_trans} \nd_cap_rot2 = {d_cap_rot2}')

d_cap_rot1 = 0.6934180492183304 
d_cap_trans = 1.122269913931213 
d_cap_rot2 = 0.267243190262223


In [80]:
print (f'P1: {probability(d_rot1-d_cap_rot1, alpha_one * d_rot1 + alpha_two * d_trans)} \nP2: {probability(d_trans - d_cap_trans, alpha_three * d_cap_trans + alpha_four * (d_cap_rot1 + d_cap_rot2))} \nP3: {probability(d_rot2 - d_cap_rot2, alpha_one * d_cap_rot2 + alpha_two * d_cap_trans)}')

P1: 1.0789502319609352 
P2: 0.39293012047112497 
P3: 0.7908997122945444
